In [1]:
## Load libraries

import pandas as pd
import numpy as np
import gensim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.preprocessing import MultiLabelBinarizer


%load_ext autoreload
%autoreload 2

In [2]:
og_glove_file = r"/Users/anshuman/UMass/CS 682/Project/glove.6B/glove.6b.300d.txt" ## glove text file that you download from internet, 6b words 300d,
new_glove_file =r"glove6b300d.bin"
movie_data_file =r"ml-1m/movies.dat" #movie data file
user_data_file =r"ml-1m/users.dat" #user data file
rating_data_file = r"ml-1m/ratings.dat" #rating data file

In [3]:
##Run this only once ever, if you have skip this and go to the next cell
## Insert your glove location below.
## This is done to convert .txt glove to binary format for easier loading.
glove_file = datapath(og_glove_file)
tmp_file = get_tmpfile("glove.txt")
_ = glove2word2vec(glove_file, tmp_file)
model = KeyedVectors.load_word2vec_format(tmp_file)
model.save_word2vec_format(new_glove_file,binary=True)

In [4]:
 ##load binary glove model
glove_model = gensim.models.KeyedVectors.load_word2vec_format(new_glove_file,binary=True)
##get vocabulary
vocab = list(glove_model.wv.vocab.keys())

/Users/anshuman/anaconda3/envs/cs682/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


In [5]:
## loading the dataset into dataframes

movieData = pd.read_csv(movie_data_file,sep="::",names=["MovieID","Name","Genres"],engine='python')
userData = pd.read_csv(user_data_file,sep="::",names=["UserID","Gender","Age","Occupation","Zipcode"],engine='python')
ratingData = pd.read_csv(rating_data_file,sep="::",names=["UserID","MovieID","Rating","Timestamp"],engine='python')


In [6]:
## movie data preprocessing  - getting a dictionary of movie name and id pairs,
##creating genrelist, and reseting movieData dataframe

movieNameData = movieData["Name"].unique()
movieNameDict = {movieNameData[i]:i for i in range(0,movieNameData.shape[0])}
genreList = ["Action","Adventure","Animation","Children's","Comedy","Crime","Documentary","Drama","Fantasy","Film-Noir",
              "Horror","Musical","Mystery","Romance","Sci-Fi","Thriller","War","Western"]
genres = movieData["Genres"].apply(lambda x : x.split("|"))
movieData.reset_index(inplace=True,drop=True)


In [7]:
## Create movie data vector based on embeddings or without.
## genre data is on hot encoded since each film could have one or more genre. Array size of the one hot encoding is 18 
## structure : movieid:genredata for no  word2vec embedding
## structure : movie_name_embedding:genredata for no  word2vec embedding

### In MovieEncodingEmbedding(), case-sensitivity is removed and also all characters which are not alpha numeric are removed.
### If the word isn't part of glove vocab, a 0 vector is used in its stead.
# def MovieEncodingNoEmbedding(movieid):
    
#     movieDataIDRow = movieData.loc[movieData["MovieID"]==movieid]
#     movieDataMovieID = movieNameDict[movieDataIDRow.Name.values[0]]
#     movieDataGenreEncoding = [1 if i in movieDataIDRow.Genres.values[0].split("|") else 0  for i in genreList]
#     data = [movieDataMovieID]+movieDataGenreEncoding
#     return np.array(data)

# def MovieEncodingEmbedding(movieid):
#     movieDataIDRow = movieData.loc[movieData["MovieID"]==movieid]
# #     print("Movie name", movieDataIDRow.Name)
#     movieDataMovieID = ''.join([char for char in movieDataIDRow.Name.values[0].lower() if (char.isalpha() or char==" ")]).split(" ")
#     movieNameEmbedding = np.mean([glove_model[word] if (word in vocab) else np.zeros((1,300)) for word in movieDataMovieID],axis=0)                            
#     movieDataGenreEncoding = np.array([1 if i in movieDataIDRow.Genres.values[0].split("|") else 0  for i in genreList])
#     data = np.concatenate((movieNameEmbedding, movieDataGenreEncoding.reshape(1,movieDataGenreEncoding.shape[0])), axis=None)
#     return data.reshape(1,data.shape[0])

from data_preparation import MovieEncodingEmbedding

print(movieData.shape)
numRows = movieData.shape[0]
# print(movieData, movieData.loc[0][0], movieData.loc[0][1], movieData.loc[0][2])
movie_embeddings = np.zeros((3953, 318))
for rowNum in range(numRows):
    movieID = movieData.loc[rowNum]['MovieID']
#     print(rowNum, movieID)
    movie_embeddings[movieID] = MovieEncodingEmbedding(movieID, movieData, genreList, glove_model, vocab)

print("Movie Embeddings", movie_embeddings.shape)

(3883, 3)
Movie Embeddings (3953, 318)


In [8]:
from data_preparation import UserSequences

## get user Sequences
userSequences = UserSequences(ratingData,-1)

27


In [9]:
print(len(userSequences))
print(len(userSequences[0]))
print(userSequences[99][1])

5421
27
(3873,)


In [10]:
## get user-movie encodings for the data with embedding flag false and write it to file.
## In every file each line will contain
##userid::gender::userDataAge::userDataOccupation::userDataZipCode::movieid::(one hot genre encoding of len 18[each separated by ::]):rating 
# data = UserMoviePairEncodingBatch(userSequences,False)

# with open("usermovie.dat","w") as f:
#     for i in range(len(data)):
#         for j in range(data[i].shape[0]):
#             f.write("::".join(data[i][j]))
#             f.write("\n")
                 

In [11]:
## get user-movie encodings for the data with embedding flag false and write it to file.
## In every file each line will contain
##userid::gender::userDataAge::userDataOccupation::userDataZipCode::(movieid_embedding[len 300, separated by ::])::(one hot genre encoding of len 18[each separated by ::]):rating 

# data = UserMoviePairEncodingBatch(userSequences,True)

# with open("usermoviembeddings.dat","w") as f:
#     for i in range(len(data)):
#         for j in range(data[i].shape[0]):
#             f.write("::".join(data[i][j]))
#             f.write("\n")

In [12]:
# np.array(data).shape
# print ("data", len(data), data[0].shape)
from sklearn.model_selection import train_test_split

def fix(data):
    newData = np.empty(data.shape, dtype=int)
    for i, userSeq in enumerate(data):
        for j, tup in enumerate(userSeq):
            newData[i][j] = tup[0]
    return newData

data = fix(np.array(userSequences))
train_data, test_data = train_test_split(data, test_size=0.2)
train_data, val_data = train_test_split(train_data, test_size=0.2)

train_inp = movie_embeddings[train_data[:, :-1]]
train_tgt = train_data[:, 1:]

test_inp = movie_embeddings[test_data[:, :-1]]
test_tgt = test_data[:, 1:]

val_inp = movie_embeddings[val_data[:, :-1]]
val_tgt = val_data[:, 1:]

print ("train", train_data.shape, train_inp.shape, train_tgt.shape)
print ("val", val_data.shape, val_inp.shape, val_tgt.shape)
print ("test", test_data.shape, test_inp.shape, test_tgt.shape)            

train (3468, 27) (3468, 26, 318) (3468, 26)
val (868, 27) (868, 26, 318) (868, 26)
test (1085, 27) (1085, 26, 318) (1085, 26)


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [14]:
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

In [21]:
from model import RNNModel
from learner import Learner

device = "cpu"
model = RNNModel('RNN', 3953, 318, 200, 1)
criterion = nn.CrossEntropyLoss()
learner = Learner(model, criterion)

# Loop over epochs.
lr = 1
best_val_loss = None

# train_data = batchify(torch.Tensor(train_data), 10)
# val_data = batchify(torch.Tensor(val_data), 10)
# test_data = batchify(torch.Tensor(test_data), 10)
train_data_final = torch.Tensor(train_inp).unsqueeze(2)
val_data_final = torch.Tensor(val_inp).unsqueeze(2)
test_data_final = torch.Tensor(test_inp).unsqueeze(2)

train_tgt_final = torch.Tensor(train_tgt).long().unsqueeze(2)
val_tgt_final = torch.Tensor(val_tgt).long().unsqueeze(2)
test_tgt_final = torch.Tensor(test_tgt).long().unsqueeze(2)

print("Train", train_data_final.shape, train_tgt_final.shape)
print("Val", val_data_final.shape, val_tgt_final.shape)
print("Test", test_data_final.shape, test_tgt_final.shape)

num_epochs = 50
for epoch in range(1, num_epochs):
    print ("| epoch {:3d} |".format(epoch))
    epoch_start_time = time.time()
    learner.train(train_data_final, train_tgt_final, 3953, lr)
    val_loss = learner.evaluate(val_data_final, val_tgt_final, 3953)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
            'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                       val_loss, math.exp(val_loss)))
    print('-' * 89)
    # Save the model if the validation loss is the best we've seen so far.
    if not best_val_loss or val_loss < best_val_loss:
        with open("model.pt", 'wb') as f:
            torch.save(model, f)
        best_val_loss = val_loss
#     else if lr > 1e-3:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
#         lr /= 1.5

Train torch.Size([3468, 26, 1, 318]) torch.Size([3468, 26, 1])
Val torch.Size([868, 26, 1, 318]) torch.Size([868, 26, 1])
Test torch.Size([1085, 26, 1, 318]) torch.Size([1085, 26, 1])
| epoch   1 |
Learning Rate 1
 1000/ 3468 batches | lr 1.00 | ms/batch 2098.57 | loss  7.27 | ppl  1436.76
 2000/ 3468 batches | lr 1.00 | ms/batch 2125.62 | loss  7.02 | ppl  1122.87
 3000/ 3468 batches | lr 1.00 | ms/batch 2152.05 | loss  6.93 | ppl  1018.37
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 74.56s | valid loss  6.84 | valid ppl   932.25
-----------------------------------------------------------------------------------------
| epoch   2 |
Learning Rate 1


/Users/anshuman/anaconda3/envs/cs682/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNNModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/Users/anshuman/anaconda3/envs/cs682/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/Users/anshuman/anaconda3/envs/cs682/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


 1000/ 3468 batches | lr 1.00 | ms/batch 2216.40 | loss  6.79 | ppl   887.00
 2000/ 3468 batches | lr 1.00 | ms/batch 2208.44 | loss  6.63 | ppl   754.50
 3000/ 3468 batches | lr 1.00 | ms/batch 2118.21 | loss  6.55 | ppl   699.97
-----------------------------------------------------------------------------------------
| end of epoch   2 | time: 76.22s | valid loss  6.54 | valid ppl   690.69
-----------------------------------------------------------------------------------------
| epoch   3 |
Learning Rate 1
 1000/ 3468 batches | lr 1.00 | ms/batch 2116.71 | loss  6.46 | ppl   639.42
 2000/ 3468 batches | lr 1.00 | ms/batch 2158.68 | loss  6.33 | ppl   559.04
 3000/ 3468 batches | lr 1.00 | ms/batch 2128.53 | loss  6.29 | ppl   539.55
-----------------------------------------------------------------------------------------
| end of epoch   3 | time: 74.92s | valid loss  6.42 | valid ppl   612.12
-----------------------------------------------------------------------------------------


 1000/ 3468 batches | lr 1.00 | ms/batch 2107.28 | loss  5.11 | ppl   166.04
 2000/ 3468 batches | lr 1.00 | ms/batch 2106.73 | loss  5.04 | ppl   154.41
 3000/ 3468 batches | lr 1.00 | ms/batch 2095.57 | loss  5.05 | ppl   155.40
-----------------------------------------------------------------------------------------
| end of epoch  18 | time: 73.85s | valid loss  6.38 | valid ppl   591.39
-----------------------------------------------------------------------------------------
| epoch  19 |
Learning Rate 1
 1000/ 3468 batches | lr 1.00 | ms/batch 2136.47 | loss  5.08 | ppl   161.56
 2000/ 3468 batches | lr 1.00 | ms/batch 2100.19 | loss  5.00 | ppl   148.96
 3000/ 3468 batches | lr 1.00 | ms/batch 2098.22 | loss  5.02 | ppl   150.87
-----------------------------------------------------------------------------------------
| end of epoch  19 | time: 74.17s | valid loss  6.38 | valid ppl   591.11
-----------------------------------------------------------------------------------------


KeyboardInterrupt: 